Notebook to classify 3k mouse brain cells. used highly variable genes and visualised gene expression in clusters

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import matplotlib.colors as mcolors
from matplotlib import rcParams
import numpy as np
import seaborn as sns
import scanpy as sc
import pandas as pd
import glmpca
import os
sc.set_figure_params(dpi=70, color_map = 'viridis_r')
FIGSIZE=(10,10)
rcParams['figure.figsize']=FIGSIZE
sc.settings.verbosity = 3
sc.logging.print_header()

In [ ]:
# plt.style.use('_mpl-gallery-nogrid')
data = sc.read_h5ad('dataset.h5ad')
print(f"var: {data.var.shape}\n obs: {data.obs.shape}\n X: {data.X.shape}\n uns: {len(data.uns)}")
data.var_names_make_unique()
sc.pl.highest_expr_genes(data, n_top=20, )
sc.pp.filter_cells(data, min_genes=200)
sc.pp.filter_genes(data, min_cells=20)
print(f"var: {data.var.shape}\n obs: {data.obs.shape}\n X: {data.X.shape}\n uns: {len(data.uns)}")
nmg = [item for item in data.var_names if item.startswith('MT-')]
print(f" Number of genes: {len(data.var_names)}\n Number of mitocondrial genes: {len(nmg)}")
data.var['mt'] = data.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
# since there are no found mitocondrial genes meaning we dont quality control using these
sc.pp.calculate_qc_metrics(data, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(data, ['n_genes_by_counts', 'total_counts', 'n_genes'], jitter=0.4, multi_panel=True)

In [ ]:
def pds(self):
    print(f"var: {data.var.shape}\n obs: {data.obs.shape}\n X: {data.X.shape}\n uns: {len(data.uns)}")
    print(data.var)
    print("\n Observation: ")
    print(data.obs)
    #print all the uns data clearly
    print("\n uns: ")
    print(data.uns)

pds(data)


In [ ]:
sc.pl.scatter(data, x='total_counts', y='pct_dropout_by_counts')
sc.pl.scatter(data, x='total_counts', y='n_genes_by_counts')
sc.pl.scatter(data, x='mean_counts', y='n_cells')

In [ ]:
data = data[data.obs.n_genes_by_counts < 4000, :]
data = data[data.obs.pct_counts_mt < 5, :]
# data = data[data.obs.total_counts > 35000, :]

pds(data)

In [ ]:
sc.pp.normalize_total(data, target_sum=1e4)

In [ ]:
sc.pp.log1p(data)

In [ ]:
sc.pp.highly_variable_genes(data, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=2000)
print(data.var['highly_variable'].sum())

In [ ]:
sc.pl.highly_variable_genes(data)

In [ ]:
data.raw = data
# data_hv = data[:, data.var.highly_variable]
data_hv = data
sc.pp.regress_out(data_hv, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(data_hv, max_value=10)

In [ ]:
sc.tl.pca(data_hv, svd_solver='arpack')
sc.pl.pca(data_hv)
sc.pl.pca_variance_ratio(data_hv, log=True)
print(np.min(data_hv.X), np.max(data_hv.X))
generalisedPCA = glmpca.glmpca(np.exp((data_hv.X)/10), 10, fam='poi', verbose=True)
print (generalisedPCA)

In [ ]:
print(generalisedPCA.keys())
print(generalisedPCA['factors'].shape,
generalisedPCA['loadings'].shape,
generalisedPCA['coefX'].shape,
generalisedPCA['dev'].shape)
factors= generalisedPCA["factors"]
scatter(factors[:,0],factors[:,1])
scatter(factors[:,1],factors[:,2])
scatter(factors[:,2],factors[:,3])
scatter(factors[:,3],factors[:,4])
# generalisedPCA['coefZ'].shape
# generalisedPCA['glmpca_family'].shape

In [ ]:
def deviance_entropy(X, y, model):
    return 2*metrics.log_loss(y, model.predict_proba(X), normalize=False)

import deviance

In [ ]:
explained_dev = deviance.explained_deviance(generalisedPCA['factors'], np.exp((data_hv.X)/10))
print(explained_dev)

In [ ]:
pds(data_hv)
#write which cluster each cell belongs to in a csv file
data_hv.obs.to_csv('cluster.csv')

In [ ]:
sc.pp.neighbors(data_hv, n_neighbors=10, n_pcs=40)
sc.tl.leiden(data_hv, resolution=0.5)
sc.tl.paga(data_hv) #In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running
sc.pl.paga(data_hv, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(data_hv, init_pos='paga')

In [ ]:
pds(data_hv)

In [ ]:
sc.tl.umap(data_hv)
sc.pl.umap(data_hv, color=['Frem2', 'Tspyl2', 'Meg3'])
sc.pl.umap(data_hv, color=['Frem2', 'Tspyl2', 'Meg3'], use_raw=False)
sc.pl.umap(data_hv, color=['leiden'])

In [ ]:
with open('output.txt', 'w') as f:
    # Print the values of data.x and data.var into the file
    f.write(str(data_hv.X))
    f.write(str(data_hv.var))
    f.write(str(data_hv.obs))
    f.write(str(data_hv.uns))

In [ ]:
#simple glmpca implementation 
from numpy import array,exp,random,repeat
from matplotlib.pyplot import scatter
from glmpca import glmpca
mu= exp(random.randn(20,100))
mu[range(10),:] *= exp(random.randn(100))
clust= repeat(["red","black"],10)
Y= random.poisson(mu)
res= glmpca(Y.T, 2)
factors= res["factors"]
scatter(factors[:,0],factors[:,1],c=clust)